In [1]:
import weakref
import ctypes

### we can use weak key dictionary instead of normal dictionary to avoid memory leaks

#### we can handle objects with slots (cause we store in descriptors)

#### but this will work only with hashable objects (we cannot have unhashable objects as keys in dict)

#### so this is a good approach for hashable objects

In [2]:
class IntegerValue:
    def __init__(self):
        self.values = weakref.WeakKeyDictionary()
        
    def __set__(self, instance, value):
        self.values[instance] = int(value)
        
    def __get__(self, instance, owner_class):
        if instance is None: 
            return self
        else:
            return self.values.get(instance)

In [3]:
class Point:
    x = IntegerValue()

In [4]:
p = Point()
print(hex(id(p)))

0x25e8a2ef7c8


In [5]:
p.x = 100.3
p.x

100

In [6]:
Point.x.values.keyrefs()

[<weakref at 0x0000025E8A311048; to 'Point' at 0x0000025E8A2EF7C8>]

In [7]:
del p

In [8]:
Point.x.values.keyrefs()

[]

### Let's try a solution to hashability problem

#### let's try to use an id of the object as a key in a standard dict

In [9]:
class IntegerValue:
    def __init__(self):
        self.values = {}
        
    def __set__(self, instance, value):
        print("__set__ method called...")
        self.values[id(instance)] = int(value)
        
    def __get__(self, instance, owner_class):
        if instance is None: 
            return self
        else:
            return self.values.get(id(instance))

In [10]:
class Point:
    x = IntegerValue()
    
    def __init__(self, x): #using the class attribute setting it to some value,
        #it's gonna call the set method, just like we had with properties
        self.x = x
        
    def __eq__(self, other):
        return isinstance(other, Person) and self.x == other.x
    
    

In [11]:
p = Point(20.45)

__set__ method called...


In [12]:
print(p.x)

20


In [13]:
p.x = 23
p.x

__set__ method called...


23

In [14]:
id(p), Point.x.values

(2605068765512, {2605068765512: 23})

In [15]:
def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [16]:
p_id = id(p)

In [17]:
ref_count(p_id)

1

In [18]:
del p

In [19]:
ref_count(p_id) # will have nonsence here cause object was garbage collected

0

#### we will have a dead entry in our dict

- so it's a memory leak
- and python can reuse this address (highly unlikely but possible)

In [20]:
Point.x.values

{2605068765512: 23}

### we need a way to determine if the object was destroyed and remove it in this case

weak references are aware if object is destroyed

In [21]:
p = Point(9)
weak = weakref.ref(p)

__set__ method called...


In [22]:
del p

In [23]:
weak #knows it's dead

<weakref at 0x0000025E8A321688; dead>

we can hook into this functionality

In [24]:
def obj_destroyed(obj):
    print(f"{obj} has been destroyed.")

In [25]:
p = Point(99)
w = weakref.ref(p, obj_destroyed) # add callback function

__set__ method called...


In [26]:
del p

<weakref at 0x0000025E8A31E638; dead> has been destroyed.


### we will store the value as  the weak reference to the object plus the value associating with that

In [27]:
class IntegerValue:
    def __init__(self):
        self.values = {}
        
    def __set__(self, instance, value):
        self.values[id(instance)] = (weakref.ref(instance,self._remove_object), int(value))
        
    def __get__(self, instance, owner_class):
        if instance is None: 
            return self
        else:
            value_tuple = self.values.get(id(instance))
            return value_tuple[1]
    def _remove_object(self, weak_ref):
        print(f"removing dead entry for {weak_ref}") # first make sure that callback function is actually being called

In [28]:
class Point:
    x = IntegerValue()

In [29]:
p1 = Point()
p2 = Point()

In [30]:
p1.x, p2.x = 23.4, 12.3

In [31]:
p1.x, p2.x

(23, 12)

In [32]:
ref_count(id(p1)), ref_count(id(p2))

(1, 1)

In [33]:
del p1 # self._remove_object was called

removing dead entry for <weakref at 0x0000025E8A345F98; dead>


#### let's implement _remove_object

In [40]:
class IntegerValue:
    def __init__(self):
        self.values = {}
        
    def __set__(self, instance, value):
        self.values[id(instance)] = (weakref.ref(instance,self._remove_object), int(value))
        
    def __get__(self, instance, owner_class):
        if instance is None: 
            return self
        else:
            value_tuple = self.values.get(id(instance))
            return value_tuple[1]
    def _remove_object(self, weak_ref):
        reverse_lookup = [key for key, value in self.values.items() if value[0] is weak_ref]
        if reverse_lookup:
            key = reverse_lookup[0]
            del self.values[key]


In [41]:
class Point:
    x = IntegerValue()

In [42]:
p1 = Point()
p2 = Point()

In [43]:
p1.x, p2.x = 23.4, 12.3

In [44]:
Point.x.values

{2605082924552: (<weakref at 0x0000025E8AFD2CC8; to 'Point' at 0x0000025E8B0AD608>,
  23),
 2605082924488: (<weakref at 0x0000025E8AFC8598; to 'Point' at 0x0000025E8B0AD5C8>,
  12)}

In [45]:
del p1

In [46]:
Point.x.values

{2605082924488: (<weakref at 0x0000025E8AFC8598; to 'Point' at 0x0000025E8B0AD5C8>,
  12)}

In [47]:
del p2

In [48]:
Point.x.values

{}

this solves the hashability problem and cleaning problem. 

#### but

when we create weak references to the object, weak reference objects are stored in the instance itself in a property called weakref 

In [58]:
class Person:
    pass
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

`__weakref__ is technically a data descriptor`

In [59]:
hasattr(Person.__weakref__, "__get__"), hasattr(Person.__weakref__, "__set__")

(True, True)

In [60]:
p = Person()

In [61]:
p.__dict__

{}

In [53]:
hasattr(p, "__weakref__")

True

In [54]:
print(p.__weakref__)

None


In [55]:
w = weakref.ref(p)
print(p.__weakref__)

<weakref at 0x0000025E8B096F98; to 'Person' at 0x0000025E8B25C788>


#### so if we use slots, the instance will no longer have this attribute

In [62]:
class Person:
    __slots__ = "name"
Person.__dict__ # __weakref__ and __dict__ are both gone

mappingproxy({'__module__': '__main__',
              '__slots__': 'name',
              'name': <member 'name' of 'Person' objects>,
              '__doc__': None})

In [63]:
p = Person()
hasattr(p, "__weakref__")

False

In [64]:
p.__dict__

AttributeError: 'Person' object has no attribute '__dict__'

so we cannot create weak references to Person objects

In [65]:
w = weakref.ref(p)

TypeError: cannot create weak reference to 'Person' object

#### so we need to add weakref to slots

In [66]:
class Person:
    __slots__ = "name", "__weakref__"
p = Person()

In [68]:
weakref.ref(p)

<weakref at 0x0000025E8B162F98; to 'Person' at 0x0000025E8A32C1C8>

### Another example

In [69]:
class ValidString:
    def __init__(self, min_length=0, max_length=255):
        self.data = {}
        self._min_length = min_length
        self._max_length = max_length
        
    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise ValueError('Value must be a string.')
        if len(value) < self._min_length:
            raise ValueError(
                f'Value should be at least {self._min_length} characters.'
            )
        if len(value) > self._max_length:
            raise ValueError(
                f'Value cannot exceed {self._max_length} characters.'
            )
        self.data[id(instance)] = (weakref.ref(instance, self._finalize_instance), 
                                   value
                                  )
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            value_tuple = self.data.get(id(instance))
            return value_tuple[1]  
        
    def _finalize_instance(self, weak_ref):
        reverse_lookup = [key for key, value in self.data.items()
                         if value[0] is weak_ref]
        if reverse_lookup:
            # key found
            key = reverse_lookup[0]
            del self.data[key]

In [70]:
class Person:
    __slots__ = "__weakref__", 
    
    first_name = ValidString(1, 100)
    last_name = ValidString(1, 100)
    
    def __eq__(self, other):
        return (
            isinstance(other, Person) and 
            self.first_name == other.first_name and
            self.last_name == other.last_name
        )
    
class BankAccount:
    __slots__ = "__weakref__", 
    
    account_number = ValidString(2, 255)
    
    def __eq__(self, other):
        return (
            isinstance(other, BankAccount) and 
            self.account_number == other.account_number 
        )

In [71]:
p1 = Person()
p1.first_name = ""

ValueError: Value should be at least 1 characters.

In [73]:
p2 = Person()

In [74]:
p1.first_name, p1.last_name = 'Guido', 'van Rossum'
p2.first_name, p2.last_name = 'Raymond', 'Hettinger'

In [75]:
b1, b2 = BankAccount(), BankAccount()

In [76]:
b1.account_number, b2.account_number = 'Savings', 'Checking'

In [77]:
p1.first_name, p1.last_name

('Guido', 'van Rossum')

In [78]:
b1.account_number, b2.account_number

('Savings', 'Checking')

In [79]:
Person.first_name.data

{2605084520664: (<weakref at 0x0000025E8B1711D8; to 'Person' at 0x0000025E8B2330D8>,
  'Guido'),
 2605084522824: (<weakref at 0x0000025E8B171A98; to 'Person' at 0x0000025E8B233948>,
  'Raymond')}

In [80]:
BankAccount.account_number.data

{2605084521096: (<weakref at 0x0000025E8B171458; to 'BankAccount' at 0x0000025E8B233288>,
  'Savings'),
 2605084523784: (<weakref at 0x0000025E8B171F48; to 'BankAccount' at 0x0000025E8B233D08>,
  'Checking')}

In [81]:
del p1
del p2
del b1
del b2

In [82]:
Person.first_name.data 
#something went wrong - we had an exception when we tried p1.first_name = ""
# and jupiter hold to that exception
#if we used try-except, jupiter wouldn't hold to p1 and we had empty dict here

{2605084520664: (<weakref at 0x0000025E8B1711D8; to 'Person' at 0x0000025E8B2330D8>,
  'Guido')}